# Manybody Potential Expressions

This notebook uses symbolic differentiation to derive expressions for common manybody potentials defined in terms of the following functions:

$$ \phi(r_{ij}^2, \xi_{ij}), \Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2) $$

To avoid verbosity, expressions in this notebook will use the following symbol substitution:

$$ R = R_1 = r_{ij}^2\\
R_2 = r_{ik}^2\\
R_3 = r_{jk}^2\\
\xi = \xi_{ij} $$

So that the above functions become:

$$ \phi(R, \xi), \Theta(R_1, R_2, R_3) $$


In [31]:
from sympy import *
from sympy.abc import R, xi, x
from IPython.display import display, Latex

init_printing()

R1, R2, R3 = symbols("R_{1:4}")
a, x = symbols("a, x")

Below are two classes that represent the above functions $\phi$ and $\Theta$. Their member functions `gradient()` and `hessian()` return the correct derivatives in the order expected by the `Manybody` calculator class in `matscipy`.

In [8]:
class Potential:
    def __init__(self, expression, latex):
        self.e = expression
        self.latex = latex
    def show(self):
        display(Latex(f"$${self.latex} = {latex(self.e)}$$"))
        display(Latex(f"$$\\nabla{self.latex} = {latex(self.gradient)}$$"))
        display(Latex(f"$$\\nabla^2{self.latex} = {latex(self.hessian)}$$"))

class Phi(Potential):
    def __init__(self, expression):
        super().__init__(expression, "\\phi(R, \\xi)")
        self.gradient = [(diff(self.e, v)) for v in (R, xi)]
        self.hessian = [(diff(self.e, *v)) for v in [(R, R), (xi, xi), (R, xi)]]

class Theta(Potential):
    def __init__(self, expression):
        super().__init__(expression, "\\Theta(R_1, R_2, R_3)")
        self.gradient = [(diff(self.e, v)) for v in (R1, R2, R3)]
        self.hessian = [(diff(self.e, *v)) for v in [(R1, R1), (R2, R2), (R3, R3), (R2, R3), (R1, R3), (R1, R2)]]

## Harmonic Pair Potential

Below is the definition of a harmonic pair potential, with the added contribution from the three body interaction $\xi$.

In [9]:
# Define equilibrium distance and stiffness
k, r0 = symbols("k, r_0")

energy = k / 2 * (sqrt(R) - r0)**2 + xi
harmonic = Phi(energy)
harmonic.show()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Lennard-Jones Pair Potential

LJ works nicely because of even powers that play nice with the definition in terms of the *squared distance*.

In [10]:
# Define characteristic distance and energy
eps, sigma = symbols("\\epsilon, \\sigma")

energy = 4 * eps * (sigma**12 / R**6 - sigma**6 / R**3) + xi
lj = Phi(energy)
lj.show()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Harmonic Angle Potential

Three-body potential that enforces an angle stiffness between two bonds.

$$ \Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2) = \frac{1}{2} k_\Theta (\Theta_{ijk} - \Theta_0)^2 = E(a)$$

with 

$$ E =  \frac{1}{2} k_\Theta (a - \Theta_0)^2$$
$$ a = \arccos(x)$$

The gradient with respect to distance/ squared distance $r_x$/$R_x=r_x^2$ with $x=\{ij, ik, jk\}$ reads

$$\frac{\partial\Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2)}{\partial r_x} = \frac{\partial \Theta}{\partial x} \frac{\partial x}{\partial r_x}$$

$$\frac{\partial\Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2)}{\partial R_x} = \frac{\partial \Theta}{\partial x} \frac{\partial x}{\partial r_x} \frac{\partial r_x}{\partial R_x}$$

The Hessian with respect to distance/ squared distance reads

$$
\frac{\partial^2 \Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2)}{\partial r_y \partial r_x} 
= 
\frac{\partial^2 \Theta}{\partial y \partial x}\frac{\partial y}{\partial r_y} \frac{\partial x}{\partial r_x}
+
\frac{\partial \Theta}{\partial r_x} \frac{\partial^2 x}{\partial r_y \partial r_x}
$$

$$
\frac{\partial^2 \Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2)}{\partial R_y \partial R_x} 
= 
\left(
\frac{\partial^2 \Theta}{\partial y \partial x}\frac{\partial y}{\partial r_y} \frac{\partial x}{\partial r_x}
+
\frac{\partial \Theta}{\partial r_x} \frac{\partial^2 x}{\partial r_y \partial r_x}
\right)
\frac{\partial r_x}{\partial R_x}\frac{\partial r_y}{\partial R_y}
+
\frac{\partial \Theta}{\partial x}\frac{\partial x}{\partial r_x} \frac{\partial^2 r_x}{\partial R_y \partial R_x}
$$ 

Additional we have the derivatives
$$
\frac{\partial \Theta}{\partial x} = \frac{\partial \Theta}{\partial a} \frac{\partial a}{\partial x}
$$

$$
\frac{\partial^2 \Theta}{\partial y \partial x} 
=
\frac{\partial^2 \Theta}{\partial a^2} \frac{\partial a}{\partial y} \frac{\partial a}{\partial x}
+
\frac{\partial \Theta}{\partial a} \frac{\partial^2 a}{\partial y \partial x}
$$

In [32]:
k_theta, theta0 = symbols("k_\\theta, \\theta_0")

cos_angle = (R1 + R3 - R2)/(2 * sqrt(R1 * R3))

# Print derivatives of angle cosine
Theta(cos_angle).show()

# Derivatives with respect to x 
E = 0.5 * k_theta * (a - theta0)**2
dTheta_da = Theta(E)
dTheta_da.gradient = diff(E, a)
dTheta_da.hessian = diff(E, *(a, a))
dTheta_da.show()

a = acos(x)
da_dx = Theta(a)
da_dx.gradient = diff(a, x)
da_dx.hessian = diff(a, *(x, x))
da_dx.show()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Stillinger-Weber Potential

The functional form of the Stillinger-Weber potential reads

$$ \Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2) = \frac{1}{2} k_\Theta (\Theta_{ijk} - \Theta_0)^2 = E(a)$$

with 

$$ E =  \frac{1}{2} k_\Theta (a - \Theta_0)^2$$
$$ a = \arccos(x)$$

The gradient with respect to distance/ squared distance $r_x$/$R_x=r_x^2$ with $x=\{ij, ik, jk\}$ reads

$$\frac{\partial\Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2)}{\partial r_x} = \frac{\partial \Theta}{\partial x} \frac{\partial x}{\partial r_x}$$

$$\frac{\partial\Theta(r_{ij}^2, r_{ik}^2, r_{jk}^2)}{\partial R_x} = \frac{\partial \Theta}{\partial x} \frac{\partial x}{\partial r_x} \frac{\partial r_x}{\partial R_x}$$

The Hessian with respect to distance/ squared distance reads